In [1]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH16-Evaluations")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH16-Evaluations


# RAG 성능 테스트를 위한 함수 정의

In [2]:
from myrag import PDFRAG


# 질문에 대한 답변하는 함수를 생성
def ask_question_with_llm(llm):
    # PDFRAG 객체 생성
    rag = PDFRAG(
        "data/SPRI_AI_Brief_2023년12월호_F.pdf",
        llm,
    )

    # 검색기(retriever) 생성
    retriever = rag.create_retriever()

    # 체인(chain) 생성
    rag_chain = rag.create_chain(retriever)

    def _ask_question(inputs: dict):
        # 질문에 대한 컨텍스트 검색
        context = retriever.invoke(inputs["question"])
        # 검색된 문서들을 하나의 문자열로 결합
        context = "\n".join([doc.page_content for doc in context])
        # 질문, 컨텍스트, 답변을 포함한 딕셔너리 반환
        return {
            "question": inputs["question"],
            "context": context,
            "answer": rag_chain.invoke(inputs["question"]),
        }

    return _ask_question

In [3]:
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama

gpt_chain = ask_question_with_llm(ChatOpenAI(model="gpt-4o-mini", temperature=0))
gpt_chain2 = ask_question_with_llm(ChatOpenAI(temperature=0))

In [4]:
from langchain_teddynote.evaluator import OpenAIRelevanceGrader
from langchain_openai import ChatOpenAI


rq_grader = OpenAIRelevanceGrader(
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0), target="retrieval-question"
).create()

ra_grader = OpenAIRelevanceGrader(
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0), target="retrieval-answer"
).create()

In [5]:
rq_grader.invoke(
    {
        "input": "삼성전자가 자체 개발한 생성형 AI 의 이름은?",
        "context": "삼성전자 AI 는 빅스비에요",
    }
)

GradeRetrievalQuestion(score='no')

In [6]:
ra_grader.invoke(
    {
        "input": "삼성전자가 자체 개발한 생성형 AI 는 가우스 입니다.",
        "context": "삼성전자 AI 는 빅스비에요",
    }
)

GradeRetrievalAnswer(score='no')

# 관련성(Relevance) 평가를 종합하는 Summary Evaluator

In [7]:
from typing import List
from langsmith.schemas import Example, Run
from langsmith.evaluation import evaluate


def relevance_score_summary_evaluator(runs: List[Run], examples: List[Example]) -> dict:
    rq_scores = 0  # 질문 관련성 점수
    ra_scores = 0  # 답변 관련성 점수

    for run, example in zip(runs, examples):
        question = example.inputs["question"]
        context = run.outputs["context"]
        prediction = run.outputs["answer"]

        # 질문 관련성 평가
        rq_score = rq_grader.invoke(
            {
                "input": question,
                "context": context,
            }
        )
        # 답변 관련성 평가
        ra_score = ra_grader.invoke(
            {
                "input": prediction,
                "context": context,
            }
        )

        # 관련성 점수 누적
        if rq_score.score == "yes":
            rq_scores += 1
        if ra_score.score == "yes":
            ra_scores += 1

    # 최종 관련성 점수 계산 (질문 관련성과 답변 관련성의 평균)
    final_score = ((rq_scores / len(runs)) + (ra_scores / len(runs))) / 2

    return {"key": "relevance_score", "score": final_score}

In [9]:
# 평가 실행
dataset_name = "RAG_EVAL_DATASET"

experiment_result1 = evaluate(
    gpt_chain,
    data=dataset_name,
    summary_evaluators=[relevance_score_summary_evaluator],
    experiment_prefix="SUMMARY_EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "GPT-4o-mini 사용: summary_evaluator 를 활용한 relevance 평가",
    },
)

# 평가 실행
experiment_result2 = evaluate(
    gpt_chain2,
    data=dataset_name,
    summary_evaluators=[relevance_score_summary_evaluator],
    experiment_prefix="SUMMARY_EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "GPT-4o 사용: summary_evaluator 를 활용한 relevance 평가",
    },
)

View the evaluation results for experiment: 'SUMMARY_EVAL-0b24e4c2' at:
https://smith.langchain.com/o/9b141874-d093-4103-946d-7bc247255f98/datasets/899fb1c5-744d-4f35-a48e-68fe78d807f1/compare?selectedSessions=5a7d5350-5448-4755-bbcb-51485126291b




0it [00:00, ?it/s]

View the evaluation results for experiment: 'SUMMARY_EVAL-8016af08' at:
https://smith.langchain.com/o/9b141874-d093-4103-946d-7bc247255f98/datasets/899fb1c5-744d-4f35-a48e-68fe78d807f1/compare?selectedSessions=bd28fb31-f234-4b28-9c66-27ee21dbb235




0it [00:00, ?it/s]